In [155]:
using Plots, LinearAlgebra

In [195]:
module MyFiniteDiff
using LinearAlgebra
function _e(T, i, n)
    v = zeros(T, n)
    v[i] = one(T)
    return v
end

"Find the numerical gradient of f : R^2 -> R using forward or centered differences"
function grad_finite_diff(x, f, h, type_)   
    n = size(x, 1)
    grad = similar(x)
    
    fx = f(x)
    
    e(i) = _e(eltype(x), i, n)
    
    for i in 1:n
        if type_ == :fw
            grad[i] = (f(x + h*e(i)) - fx) / h
        end
        if type_ == :c
            grad[i] = (f(x + h*e(i)) - f(x - h*e(i))) / 2h
        end
    end
    
    return grad
end

function hess_finite_diff(x, f, h)
    n = size(x, 1)
    grad = similar(x, n, n)
    
    e(i) = _e(eltype(x), i, n)
    
    for i in 1:n
        for j in 1:n
            # Just calculate the upper triangle
            if i < j
                grad[i, j] = (
                    f(x + h*e(i) + h*e(j)) - f(x + h*e(i)) - f(x + h*e(j)) + f(x)
                ) / (h^2)
            end
            if i == j
                grad[i, i] = (
                    f(x + h*e(i)) - 2*f(x) + f(x - h*e(i))
                ) / (h^2)
            end
        end
    end
    
    # And then copy the upper part into the lower part
    return Symmetric(grad)
end

# TODO(Friday): Jacobian, Hessian as Jacobian, Finite differences in Netwon Method
    
end
import Main.MyFiniteDiff

@assert MyFiniteDiff.grad_finite_diff([3.], x -> x[1]^2, sqrt(eps()), :fw) ≈ [6]
@assert MyFiniteDiff.grad_finite_diff([3.], x -> x[1]^2, sqrt(eps()), :c) ≈ [6]
@assert MyFiniteDiff.grad_finite_diff([float(pi)], x -> sin(x[1]), sqrt(eps()), :fw) ≈ [-1]
@assert MyFiniteDiff.grad_finite_diff([float(pi)], x -> sin(x[1]), sqrt(eps()), :c) ≈ [-1]

# Fails with sqrt(eps()). Big numerical cancellation problems
@assert isapprox(MyFiniteDiff.hess_finite_diff([3., 2.], x -> x[1]^2 + x[2]^2, sqrt(eps())*norm([3., 2.])), [2. 0.; 0. 2.], atol=1.2)
# much more precise with a bigger step
@assert isapprox(MyFiniteDiff.hess_finite_diff([3., 2.], x -> x[1]^2 + x[2]^2, 1e-6), [2. 0.; 0. 2.], atol=5e-3)